In [1]:
import tensorflow as tf
from tensorflow.keras import layers


2022-11-14 04:45:41.438626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 04:45:41.832636: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
examples = tf.random.uniform((1,300,300,3), dtype=tf.float32, minval=0, maxval=255)

In [57]:
def down_sample_blk(out_channels):
    blk = []
    for _ in range(2):
        blk.append(tf.keras.layers.Conv2D(out_channels,
                             kernel_size=3, padding='same'))
        blk.append(tf.keras.layers.BatchNormalization())
        blk.append(tf.keras.layers.ReLU())
    blk.append(tf.keras.layers.MaxPool2D(pool_size=2))
    return tf.keras.models.Sequential(blk)

def vgg_block(num_convs, num_channels):
    blk = tf.keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(
            tf.keras.layers.Conv2D(num_channels, kernel_size=3,
                                   padding='same', activation='relu'))
    blk.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding="same"))
    return blk
def conv4_3():
    return tf.keras.models.Sequential([
        vgg_block(2,64),
        vgg_block(2,128),
        vgg_block(3,256),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu'),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu'),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu')
    ])

def fc_7():
    return tf.keras.models.Sequential([
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu'),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu'),
        tf.keras.layers.Conv2D(512, kernel_size=3,
                                   padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(3,1,padding='same'),
        tf.keras.layers.Conv2D(1024,3,padding="same"),
        tf.keras.layers.Conv2D(1024,1,padding="same"),
    ])

def conv8_2():
    return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256,1,padding="same"),
    tf.keras.layers.Conv2D(512,3,padding="same", strides=2),
    ])

def conv9_2():
    return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128,1,padding="same"),
    tf.keras.layers.Conv2D(256,3,padding="same", strides=2),
    ])
def conv10_2():
    return conv9_2()

def conv11_2():
    return tf.keras.layers.GlobalAveragePooling2D(keepdims=True)
# blks = [
#     vgg_block(2,64),
#     vgg_block(2,128),
#     vgg_block(3,256),
#     vgg_block(3,512),
#     vgg_block(3,512),

#     # tf.keras.layers.Conv2D(1024,3,padding="same"),
#     # tf.keras.layers.Conv2D(1024,1,padding="same"),

#     # tf.keras.layers.Conv2D(256,1,padding="same"),
#     # tf.keras.layers.Conv2D(512,3,padding="same", strides=2),

#     # tf.keras.layers.Conv2D(128,1,padding="same"),
#     # tf.keras.layers.Conv2D(256,3,padding="same", strides=2),

#     # tf.keras.layers.Conv2D(128,1,padding="same"),
#     # tf.keras.layers.Conv2D(256,3,padding="same", strides=2),

#     # tf.keras.layers.GlobalAveragePooling2D(keepdims=True),
# ]

blks = [
    conv4_3(),
    fc_7(),
    conv8_2(),
    conv9_2(),
    conv10_2(),
    conv11_2(),
]
vgg_blocks = tf.keras.models.Sequential(blks)
examples = tf.random.uniform((1,300,300,3), dtype=tf.float32, minval=0, maxval=255)
# vgg_blocks(examples)
for blk in blks:
    
    examples = blk(examples)
    print(examples.shape)

(1, 38, 38, 512)
(1, 19, 19, 1024)
(1, 10, 10, 512)
(1, 5, 5, 256)
(1, 3, 3, 256)
(1, 1, 1, 256)


In [25]:
layer = conv10_2()
layer(tf.random.normal((1,3,3,256)))

<tf.Tensor: shape=(1, 2, 2, 256), dtype=float32, numpy=
array([[[[ 0.25216255, -0.19294423, -0.5687745 , ...,  0.28736937,
           0.19958235, -0.48744464],
         [ 1.207684  , -1.1445202 , -0.6397885 , ...,  0.6514596 ,
          -0.2160021 , -0.68929625]],

        [[-0.22901106, -0.39233539,  1.0736238 , ..., -0.14732064,
          -0.8068203 , -0.2541527 ],
         [ 1.1012512 ,  0.63099253, -0.25680518, ..., -0.54186434,
          -0.39874637,  0.1633453 ]]]], dtype=float32)>

In [37]:
vgg_blocks.summary()

Model: "sequential_121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_116 (Sequential)  (1, 150, 150, 64)        38720     
                                                                 
 sequential_117 (Sequential)  (1, 75, 75, 128)         221440    
                                                                 
 sequential_118 (Sequential)  (1, 37, 37, 256)         1475328   
                                                                 
 sequential_119 (Sequential)  (1, 18, 18, 512)         5899776   
                                                                 
 sequential_120 (Sequential)  (1, 9, 9, 512)           7079424   
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [40]:
a = tf.keras.applications.VGG16(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1,
    classifier_activation="softmax",)
a.compute_output_shape((1,300,300,3))
# a.get_layer('block5_pool').get_config()

TensorShape([1, 9, 9, 512])

In [65]:
import math
def generate_sizes(n=5,start=0.2,end=1.05):
    sizes = []
    t = (end - start) / n
    for i in range(n):
        if i==0:
            s_0 = start
            # s_1 = start + tf.sqrt(s_0*(s_0 + t))
            # sizes = [[s_0, s_1]]
        else:
            s_0 = sizes[-1][0] + t
        s_1 = math.sqrt(s_0*(s_0 + t))
        sizes.append([s_0, s_1])
    return sizes

generate_sizes()

[[0.2, 0.2720294101747089],
 [0.37, 0.4469899327725402],
 [0.54, 0.6191930232165088],
 [0.7100000000000001, 0.7904429138147803],
 [0.8800000000000001, 0.9612491872558333]]

In [63]:
(1.05 - 0.2)/5

0.17